This demo use SHAP to visualize the effect of fine-tuning and save the plot in file. 
RUN in Colap

In [1]:
!pip install transformers
!pip install shap
!pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 18.0 MB/s 
     |████████████████████████████████| 6.6 MB 62.3 MB/s 
     |████████████████████████████████| 101 kB 16.7 MB/s 
     |████████████████████████████████| 596 kB 59.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 31.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import shap
import transformers
import torch
import numpy as np
import scipy as sp

# load a BERT sentiment analysis model
ft_tokenizer = transformers.DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
ft_model = transformers.DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
).cuda()
b_tokenizer = transformers.DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
b_model = transformers.DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased"
).cuda()

# define a prediction function
def ft_f(x):
    tv = torch.tensor([ft_tokenizer.encode(v, padding='max_length', max_length=500, truncation=True) for v in x]).cuda()
    outputs = ft_model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val
   

def b_f(x):
    tv = torch.tensor([b_tokenizer.encode(v, padding='max_length', max_length=500, truncation=True) for v in x]).cuda()
    outputs = b_model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier

In [41]:
# build an explainer using a token masker
ft_explainer = shap.Explainer(ft_f, ft_tokenizer)
b_explainer = shap.Explainer(b_f, b_tokenizer)
text = ["the film was very cool and the players were perfect"] # the sentence you need to analyze
b_shap_values = b_explainer(text)
ft_shap_values = ft_explainer(text)

In [39]:
shap.plots.text(b_shap_values)
shap.plots.text(ft_shap_values)

In [42]:
string = shap.plots.text(b_shap_values, display=False).replace("[0]", "Distilbert base") + "<br>" + shap.plots.text(ft_shap_values, display=False).replace("[0]", "Distilbert fine-tuned")

string = string.replace("margin-top: -15px;", "margin-top: -10px;") # margin between the line chart and the text
string = string.replace("dashed", "#FFFFFF") # remove upper dashd line 
string = string.replace("background: #fff", "") # title background color
string = string.replace("#ffffff", "#FBFBFE") # start of the plot color
from bs4 import BeautifulSoup
try: 
    from BeautifulSoup import BeautifulSoup
except ImportError:
    from bs4 import BeautifulSoup

html = BeautifulSoup(string)

for text in html.find_all("text", {'dominant-baseline':'bottom'}):  # remove upper words
    text.decompose()

for line in html.find_all("line", {'y1':'33'}): # remove remove upper ticks 
    line.decompose()
parsed_html = str(html)
f = open("/content/drive/MyDrive/NI2.html", "w") # the file name as htnl
f.write(parsed_html)
f.close

<function TextIOWrapper.close>